# GLOBALS AND INITIAL PARAMETERS

In [14]:
from emot import emo_unicode
from keras.callbacks import Callback, EarlyStopping, CSVLogger, ReduceLROnPlateau, ModelCheckpoint
from keras.layers import Dense, Activation, Dropout, LSTM, TimeDistributed,Masking 
from keras.models import load_model, Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
from nltk.tokenize import word_tokenize #, wordpunct_tokenize
from sklearn.model_selection import train_test_split
from time import sleep
import csv
import matplotlib
import numpy as np
import re
import sys
import string

matplotlib.use('Agg')
from matplotlib import pyplot as plt 

# settings
CHAR_BY_CHAR = True # True if doing character-by-character training
CUSTOM_NAMES = True # set as True to use CUSTOM_NAME token (and not empty str) for tagged Facebook names
MODEL_NAME = 'lstm_512_charchar_likes' # name models descriptively, e.g. with param sizes like lstm_512_hidden
PRINT_TRAIN_PROGRESS_TO_TERMINAL = False # set True to see losses change in real-time in terminal
WEIGHT_BY_REACTIONS = True
LIKES_ONLY = True

# parameters
# MAX_LEN is defined later
HIDDEN_UNITS = 512
NB_EPOCHS = 2#20
STEP_SIZE = 1
WINDOW_SIZE = 10 
LSTM_MODEL = True
 
# paths with data based on filtering done in adapted version of Facebook post scraping
STATUS_FILEPATH = "csv_data/beaverconfessions_facebook_statuses.csv"

if CUSTOM_NAMES:
    COMMENTS_FILEPATH = "csv_data/CUSTOM_NAMES/custom_name_token_comments.csv"
else:
    COMMENTS_FILEPATH = "csv_data/FILTERED_NAMES/filtered_names_comments.csv"

# symbols
CUSTOM_NAME = 'CUSTOM_NAME'
CUSTOM_NUMBER = 'CUSTOM_NUMBER'
PADDING_SYMBOL = "{"
STOP_SYMBOL = "`"

# PRE-PROCESSING (TOKENIZATION) 

In [15]:
escaped_punctuation = re.escape(string.punctuation)
print 'Escaped punctuation:', escaped_punctuation

# insert an OR pipe before each punctuation mark
xor_punctuation = '|'.join('{}{}+'.format(escaped_punctuation[x], escaped_punctuation[x+1]) for x in range(0, len(escaped_punctuation), 2))
# print 'Delimited punctuation:', xor_punctuation

# build regex with variable (order matters!)
custom_name = CUSTOM_NAME.lower()
course_number = '\d+\.\d{2,3}'
multiple_numbers = '\d+'
emoticon_pattern = '|'.join(emoticon for emoticon in emo_unicode.EMOTICONS)
space = '\s'
regex_expr = r'(' + '|'.join([custom_name, course_number, multiple_numbers, emoticon_pattern, xor_punctuation, space]) + r')'
# print '\nRegex expression:', regex_expr

def replace_digit_with_token(string): 
    return CUSTOM_NUMBER if string.isdigit() else string

def tokenize_str(string, replace_digits=True):
    """
    NLTK tokenizers (word_tokenize, wordpunct_tokenize) are insufficient, 
    as emoticons and course #s, e.g. 6.111, are important to our dataset.
    
    This is a custom tokenizer to retain such items in the vocab,
        but split up other words containing numbers/punctuation, e.g. 3pm.
        
    Note that these texts are lowercased by default prior to tokenization.
    """
    if CHAR_BY_CHAR:
        tokens = list(string)
        
    else:
        tokens = re.split(regex_expr, string.lower())

        # filter out spaces
        tokens = [token for token in tokens if token not in ["", " "]]

        # tokens with *just* digits are mapped to CUSTOM_NUMBER by default
        if replace_digits:
            tokens = map(lambda x: replace_digit_with_token(x), tokens)

    return tokens

# attempt to catch major cases in our dataset
sentence = 'CUSTOM_NAME 6.867test. @7:30pm in 54-100. The profs, they\'re coming in 3..2..1. LETS DOOO THIS!!! >;)' + STOP_SYMBOL
print '\nTest case:', tokenize_str(sentence)

Escaped punctuation: \!\"\#\$\%\&\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\]\^\_\`\{\|\}\~

Test case: ['C', 'U', 'S', 'T', 'O', 'M', '_', 'N', 'A', 'M', 'E', ' ', '6', '.', '8', '6', '7', 't', 'e', 's', 't', '.', ' ', '@', '7', ':', '3', '0', 'p', 'm', ' ', 'i', 'n', ' ', '5', '4', '-', '1', '0', '0', '.', ' ', 'T', 'h', 'e', ' ', 'p', 'r', 'o', 'f', 's', ',', ' ', 't', 'h', 'e', 'y', "'", 'r', 'e', ' ', 'c', 'o', 'm', 'i', 'n', 'g', ' ', 'i', 'n', ' ', '3', '.', '.', '2', '.', '.', '1', '.', ' ', 'L', 'E', 'T', 'S', ' ', 'D', 'O', 'O', 'O', ' ', 'T', 'H', 'I', 'S', '!', '!', '!', ' ', '>', ';', ')', '`']


In [20]:
def load_dataset_and_vocabulary(filepath, vocabulary, datatype, max_nb_tokens, reactions):
    """
    Read individual sentences into memory, and generate vocabulary.
    
    If CHAR_BY_CHAR, the vocabulary will hold single characters, 
        e.g. a-zA-Z and punctuation.
    Else, it will contain whole words and Unicode 'phrases',
        e.g. :\'(, as split by our custom tokenizer.
    """
    dt = "{}_message".format(datatype)
    
    if LIKES_ONLY:
        reaction_header = 'num_likes'
    else:
        reaction_header = 'num_reactions'
    
    with open(filepath, "rU") as csvfile:
        reader = csv.DictReader(csvfile)
        tokens = []
        
        for status in reader:
            # we distinguish between comments and statuses with header cols in the CSVs
#             if dt not in status:
#                 print 'test'
#                 # 2 rows are read at once from STATUS_FILEPATH for some reason...
#                 # handled with monkey patching 
#                 msg1, msg2 = status.items()[1]
#                 # num_reacts1, num_reacts2 = status[reaction_header] # ...?
                
#                 # append stop symbol so model has explicit marker for ending
#                 msg1 += STOP_SYMBOL
#                 msg2 += STOP_SYMBOL
                
#                 # we define elts as the unit we are training on, e.g. char vs. word 
#                 elts_in_sentence1 = tokenize_str(msg1)
#                 elts_in_sentence2 = tokenize_str(msg2)
                
#                 max_nb_tokens = max(max_nb_tokens, max(len(elts_in_sentence1), len(elts_in_sentence2)))
                 
#                 for elt in elts_in_sentence1:
#                     vocabulary.add(elt)
#                 for elt in elts_in_sentence2:
#                     vocabulary.add(elt) 
                    
#                 # build list of tokens in local memory
#                 tokens.append(elts_in_sentence1)
#                 tokens.append(elts_in_sentence2)
#             else:
            msg = status[dt]
            num_reacts = float(status[reaction_header])

            msg += STOP_SYMBOL 
            elts_in_sentence = tokenize_str(msg)
            max_nb_tokens = max(max_nb_tokens, len(elts_in_sentence))

            for elt in elts_in_sentence:
                vocabulary.add(elt)

            tokens.append(elts_in_sentence)
            reactions.append(num_reacts)
                    
    return vocabulary, tokens, max_nb_tokens, reactions
    
# load comments and statuses separately at first
max_nb_tokens = 0
vocabulary, comment_tokens, max_nb_tokens, comment_reacts = load_dataset_and_vocabulary(COMMENTS_FILEPATH, set([]), "comment", max_nb_tokens, [])
vocabulary, status_tokens, max_nb_tokens, reacts = load_dataset_and_vocabulary(STATUS_FILEPATH, vocabulary, "status", max_nb_tokens, comment_reacts)
MAX_LEN = max_nb_tokens

print 'Max # of tokens in a sentence:', MAX_LEN
# print '\nSample of 10 processed sentences:'
for status in status_tokens[:10]:
    print status
    
max_num_reacts = max(reacts)
print '\nMax # of reactions:', max_num_reacts

# create vocabulary of characters found in data
if not CHAR_BY_CHAR:
    vocabulary.add(CUSTOM_NAME)
    vocabulary.add(CUSTOM_NUMBER)
    
# these symbols did not exist in the original training set,
# so we can safely add them 
# NOTE: changed from insert(0,_), since we were adding duplicates
vocabulary.add(STOP_SYMBOL)
vocabulary.add(PADDING_SYMBOL)
vocabulary = sorted(list(vocabulary))

NB_CLASSES = len(vocabulary)
print '\nnumber of classes:', NB_CLASSES

tokens_indices = dict((t, i) for i, t in enumerate(vocabulary))
indices_tokens = {v: k for k, v in tokens_indices.iteritems()}
PADDING_INDEX = tokens_indices[PADDING_SYMBOL]
STOP_INDEX = tokens_indices[STOP_SYMBOL]
print '\nThe padding index is:', PADDING_INDEX
print '\nThe stop index is:', STOP_INDEX

input_type = "character by character" if CHAR_BY_CHAR else "word by word"
# print "For", input_type, ", we have this vocabulary:", vocabulary, "\n of size", NB_CLASSES

Max # of tokens in a sentence: 281
['#', '9', '5', '4', '1', ' ', 'I', ' ', 'h', 'a', 'd', ' ', 'f', 'e', 'e', 'l', 'i', 'n', 'g', 's', ' ', 'f', 'o', 'r', ' ', 'y', 'o', 'u', ' ', 'l', 'a', 's', 't', ' ', 'y', 'e', 'a', 'r', ',', ' ', 'b', 'u', 't', ' ', 'I', ' ', 'w', 'a', 's', ' ', 'o', 'n', 'l', 'y', ' ', 'a', ' ', 'f', 'r', 'o', 's', 'h', ' ', 't', 'h', 'e', 'n', '.', ' ', 'N', 'o', 'w', ',', ' ', 'y', 'o', 'u', "'", 'r', 'e', ' ', 'a', ' ', 's', 'e', 'n', 'i', 'o', 'r', ',', ' ', 'a', 'n', 'd', ' ', 'I', ' ', 'd', 'o', 'n', "'", 't', ' ', 'k', 'n', 'o', 'w', ' ', 'i', 'f', ' ', 'y', 'o', 'u', "'", 'd', ' ', 'b', 'e', ' ', 'w', 'i', 'l', 'l', 'i', 'n', 'g', ' ', 't', 'o', ' ', 'm', 'a', 'k', 'e', ' ', 't', 'h', 'e', ' ', 'e', 'm', 'o', 't', 'i', 'o', 'n', 'a', 'l', ' ', 'i', 'n', 'v', 'e', 's', 't', 'm', 'e', 'n', 't', '.', ' ', 'I', ' ', 'k', 'n', 'o', 'w', ' ', 'I', "'", 'd', ' ', 'b', 'e', ' ', 'w', 'i', 'l', 'l', 'i', 'n', 'g', ' ', 't', 'o', '.', '`']
['#', '9', '5', '4', '0'

# BUILD TRAINING SETS

In [43]:
# merge comments and statuses
tokens = []
tokens.extend(comment_tokens)
tokens.extend(status_tokens)
tokens = np.array(tokens)

# shuffle dataset (commented out b/c we shuffle during training anyway)
# random_permutation = np.random.permutation(len(tokens))
# tokens = tokens_arr[random_permutation]

print 'Sample tokenized sentence:'
# print tokens_arr
print tokens[0]
print

def generate_X_and_Y(sentences):
    """
    X = sub_sentences, Y = next_sub_sentences
    Y is simply X shifted over by step_size.
    We want sub sentences per sentence to create multiple samples.
    """
    sub_sentences = []
    next_sub_sentences = []
    sub_reactions = []
    for sentence_nb, sentence in enumerate(sentences):
        sentence_reacts = reacts[sentence_nb]
        for i in range(0, len(sentence) - WINDOW_SIZE, STEP_SIZE):
            sub_sentences.append(sentence[i : i+WINDOW_SIZE])
            next_sub_sentences.append(sentence[(i+STEP_SIZE) : (i+STEP_SIZE)+WINDOW_SIZE])
            sub_reactions.append(sentence_reacts)
    
    return sub_sentences, next_sub_sentences, sub_reactions

X, Y, X_reacts = generate_X_and_Y(tokens)
nb_samples = len(X)
print 'number of samples:', nb_samples
print 'Sample X:', X[0]
print 'Sample Y:', Y[0]
print 

def convert_tokens_to_int(X, Y, nb_samples):
    """
    Convert token to integer representations.
    
    Populate zero-filled label matrices with ints,
        so end result is padded and ready for training.
    """
    # any values not filled in later represent padding 
    # extra 1 represents space for the stop symbol
    input_shape = (nb_samples, MAX_LEN+1)
    X_labels = np.zeros(input_shape)
    y_labels = np.zeros(input_shape)

    for sample_nb in range(nb_samples):
        x_label = map(lambda x: tokens_indices[x], X[sample_nb]) 
        y_label = map(lambda x: tokens_indices[x], Y[sample_nb])

        X_labels[sample_nb][:len(x_label)] = x_label
        y_labels[sample_nb][:len(y_label)] = y_label
    
    return X_labels, y_labels

X_labels, y_labels = convert_tokens_to_int(X, Y, nb_samples)
#X_labels, y_labels = X, Y
print 'Sample X labels:\n', X_labels[:1]
print 'Sample Y labels:\n', y_labels[:1]
print '\nTotal # of training samples:', nb_samples

# split data into train and val sets
X_train_labels, X_val_labels, y_train_labels, y_val_labels = train_test_split(X_labels, y_labels, test_size=0.3)
num_train = len(X_train_labels)
X_train_reacts = X_reacts[:num_train]
X_val_reacts = X_reacts[num_train:]

Sample tokenized sentence:
['#', '3', '0', '5', '6', '4', ' ', 'S', 'o', 'm', 'e', 't', 'h', 'i', 'n', 'g', ' ', 's', 'o', 'm', 'e', 't', 'h', 'i', 'n', 'g', ' ', 'i', 't', "'", 's', ' ', 'w', 'h', 'a', 't', ' ', 'Y', 'O', 'U', ' ', 'm', 'a', 'k', 'e', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'e', 'x', 'p', 'e', 'r', 'i', 'e', 'n', 'c', 'e', ',', ' ', 'm', 'a', 'y', 'b', 'e', ' ', 'y', 'o', 'u', ' ', 'j', 'u', 's', 't', ' ', 'g', 'r', 'e', 'w', ' ', 'a', 'n', 'd', ' ', 'l', 'e', 'a', 'r', 'n', 'e', 'd', ' ', 'm', 'o', 'r', 'e', ' ', 'f', 'r', 'o', 'm', ' ', 'i', 't', ' ', 't', 'h', 'a', 'n', ' ', 't', 'h', 'e', 'm', '`']

number of samples: 387097
Sample X: ['#', '3', '0', '5', '6', '4', ' ', 'S', 'o', 'm']
Sample Y: ['3', '0', '5', '6', '4', ' ', 'S', 'o', 'm', 'e']



MemoryError: 

# BATCH GENERATOR

In [46]:
if PRINT_TRAIN_PROGRESS_TO_TERMINAL:
    reload(sys)

def validate_data(X, y):
    # assert at least one nonzero value in each OHE sample
    assert(False not in np.any(X>0, axis=1))
    assert(False not in np.any(y>0, axis=1))

def sampling_generator(epoch_size, batch_size, validation=False):  
    """
    Takes in labels of int values, e.g. [1. 2. 5. 9. 1. 0. 0. ...]
    These labels are already padded with 0s to a predetermined maxlen.
    We convert each sample X and y to one hot versions, and create batches.
    Batch generation is necessary to avoid MemoryError.
    """
    if validation:
        X_labels = X_val_labels 
        y_labels = y_val_labels
        X_reacts = X_val_reacts
    else:
        X_labels = X_train_labels
        y_labels = y_train_labels
        X_reacts = X_train_reacts
    
    while True:
        # hand off data in batches
        for i in range(int(epoch_size/batch_size)):
            start = i * batch_size
            end = min(start + batch_size, epoch_size)
            true_batch_size = end - start

            # fresh batch
            batch_X = []    # all one-hot inputs for batch
            batch_Y = []    # all one-hot labels for batch
            batch_shape = (true_batch_size, MAX_LEN+1, NB_CLASSES)
            
            X_label_sample = X_labels[start:end]   # batch_size number of samples
            X_label_sample_reacts = X_reacts[start:end]   

            # one hot encode
            batch_X = to_categorical(X_label_sample, num_classes=NB_CLASSES).reshape(batch_shape)
            batch_Y = to_categorical(y_labels[start:end], num_classes=NB_CLASSES).reshape(batch_shape)
            
            total_nb_one_hots = X_label_sample.size  # this is just batch_size * number of timesteps * nb_classes
    
            sample_weights = np.ones((total_nb_one_hots, 1))
            
            temp_reshaped_Xs = np.ravel(X_label_sample).reshape(-1)
            padded_positions = np.where(temp_reshaped_Xs == PADDING_INDEX)
            sample_weights[padded_positions] = 0 # 28100 x 1
            sample_weights = sample_weights.reshape(true_batch_size, MAX_LEN+1)
            print sample_weights
            
            if WEIGHT_BY_REACTIONS:
                react_weights = np.array(X_label_sample_reacts).reshape(-1,1) / max_num_reacts + 1.0
                sample_weights = np.multiply(sample_weights, react_weights)
            
            yield (batch_X, batch_Y, sample_weights)

# MODEL ARCHITECTURE

In [23]:
def build_model():
    if LSTM_MODEL:
        print('Building LSTM model...')
        model = Sequential()
        # model.add(Masking(input_shape=(MAX_LEN+1,1)))
        model.add(LSTM(512, return_sequences=True, input_shape=(MAX_LEN+1,NB_CLASSES)))
        model.add(LSTM(512, return_sequences=True))
        # model.add(Dropout(0.2))
        model.add(TimeDistributed(Dense(NB_CLASSES)))
        model.add(Activation('softmax'))
        # if we want to apply sample weights to metrics, we need to specify weighted_metrics=[list of metrics]
        model.compile(loss='categorical_crossentropy', optimizer=Adam(clipnorm=1.0), sample_weight_mode="temporal")   
    else:
        print('Building GRU model...')
        model = Sequential()
        model.add(GRU(512, return_sequences=True, input_shape=(MAX_LEN+1, NB_CLASSES)))
        model.add(Dropout(0.2))
        model.add(GRU(512, return_sequences=True))
        model.add(Dropout(0.2))
        model.add(Dense(NB_CLASSES, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=Adam(clipnorm=1.0), sample_weight_mode="temporal")

    print ('Model is made!')
    
    model.summary()
    return model


model = build_model()

Building LSTM model...
Model is made!
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 282, 512)          1243136   
_________________________________________________________________
lstm_2 (LSTM)                (None, 282, 512)          2099200   
_________________________________________________________________
time_distributed_1 (TimeDist (None, 282, 94)           48222     
_________________________________________________________________
activation_1 (Activation)    (None, 282, 94)           0         
Total params: 3,390,558
Trainable params: 3,390,558
Non-trainable params: 0
_________________________________________________________________


# CALLBACKS (bells + whistles)

In [8]:
class PlotHistory(Callback):
    def __init__(self, run_name):
        self.run_name = run_name 

    def on_train_begin(self, logs=None):
        self.epoch = []
        self.history = {}

    def on_epoch_end(self, epoch, logs=None):
        logs = logs or {}
        self.epoch.append(epoch)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)

        # create loss and perplexity plot
        loss_handles = []
        train_perp = []
        val_perp = []
        for key in self.history:           
            if key != "lr":
                l, = plt.plot(self.history[key], label=key)
                loss_handles.append(l)

        plt.title('Losses and metrics for {}'.format(self.run_name))    
        plt.ylabel('loss')
        plt.yscale('symlog')
        plt.legend(["Training Loss","Validation Loss"], fontsize=8, loc='upper right')          
        plt.savefig('{}_plot.jpg'.format(self.run_name))        
        plt.clf()
    
checkpointer = ModelCheckpoint(filepath='{}.hdf5'.format(MODEL_NAME), verbose=1, save_best_only=True)
csv_logger = CSVLogger('{}.log'.format(MODEL_NAME))
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001, verbose=1)
plot_history = PlotHistory(MODEL_NAME)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, epsilon=1e-4, min_lr=1E-6) 

# PERPLEXITY

In [9]:
def plotPerplexity(model):
    f = open(model+'.log','r')
    epoch_X = []
    train_perplex = []
    val_perplex = []

    for line in f.readlines()[1:]:
        data = line.strip().split(",")
        epoch_X.append(int(data[0]))
        train_perplex.append(2**(float(data[1])))
        val_perplex.append(2**(float(data[3])))

    fig = plt.figure()
    ax = plt.axes()

    ax.plot(epoch_X, train_perplex)
    ax.plot(epoch_X, val_perplex)
    plt.legend(['Training perplexity', 'Validation perplexity'], fontsize=8, loc='upper right')
    plt.title('Perplexities for {}'.format(model))
    plt.savefig('{}_perplexity.jpg'.format(model))
    plt.clf()

# TRAIN / LOAD MODEL

In [10]:
if PRINT_TRAIN_PROGRESS_TO_TERMINAL:
    reload(sys)

small_samples = 200
nb_train_samples = small_samples # len(X_train_labels)
nb_val_samples = small_samples # len(X_val_labels)
print 'Total training samples:', nb_train_samples
print 'Total val samples:', nb_val_samples
batch_size = 128 # nb_val_samples/2

def train(model):
    # shuffling of training data True by default
    print 'Training...'
    history = model.fit_generator(
        sampling_generator(nb_train_samples, batch_size), 
        steps_per_epoch=nb_train_samples/batch_size,
        epochs=NB_EPOCHS,
        verbose=1,
        validation_data=sampling_generator(nb_val_samples, batch_size, validation=True),
        validation_steps=nb_val_samples/batch_size,
        callbacks=[early_stopping, reduce_lr, csv_logger, checkpointer, plot_history])

    hist = history.history
    # print 'Loss:', hist['loss'][0], 'and val loss:', hist['val_loss'][0]
    print 'Full training history:\n', history

train(model)
plotPerplexity(MODEL_NAME)

StopIteration: 

In [59]:
model = load_model('{}.hdf5'.format(MODEL_NAME))

# TEXT GENERATION

In [60]:
def convert_sentence_to_ohe(sentence):
    x_label = map(lambda x: tokens_indices[x], sentence)
    confession = np.zeros(MAX_LEN+1)
    confession[:len(x_label)] = x_label
    ohe_x = to_categorical(confession, num_classes=NB_CLASSES)
    return np.expand_dims(ohe_x, axis=0)

def generate_confession(model, seed_string):
    # nb chars to preserve
    orig_len = len(seed_string) 
    window_str = seed_string
    final_str = seed_string
    
    for char_nb in range(orig_len, MAX_LEN):
        x = convert_sentence_to_ohe(window_str)
        
        # get next char
        preds = model.predict(x)[0] # otherwise wrapped in (1,maxlen+1,len(chars))
        best_tokens = np.argmax(preds, axis=1)
        print "best_tokens:",best_tokens
        
#         word = ""
#         for j in best_tokens:
#             word += indices_tokens[j]
#         print "predicted Y:",word
        
        # char_nb-1 because we want prev val in y matrix (best_tokens). In training we treat y as a shifted 
            # version of x hence offset -1 here
        if char_nb >= WINDOW_SIZE:
            next_token = best_tokens[WINDOW_SIZE-1]
        else:
            next_token = best_tokens[char_nb-1] 
        
        print "Next_token:", next_token, "is:", indices_tokens[next_token]

        # stop symbol
        if next_token == STOP_INDEX:
            break
            
        next_char = indices_tokens[next_token]
        print 'Current string:', window_str
    
        if len(window_str) == WINDOW_SIZE:
            moveConf = window_str[1:] + next_char
            window_str = moveConf
            print "\n New window_str:", window_str
        else:
            window_str += next_char
            
        final_str += next_char
        
        print "Final str:", final_str
          
    return final_str

seed_string = "#"
print generate_confession(model, seed_string)

best_tokens: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]
Next_token: 0 is: !
Current string: #
Final str: #!
best_tokens: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]
Next_token: 0 is: !
Current string: #!
Final str: #!!
best_tokens: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]
Next_token: 0 is: !
Current string: #!!
Final str: #!!!
best_tokens: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0]
Next_token: 0 is: !
Current string: #!!!
Final str: #!!!!
best_tokens: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

KeyboardInterrupt: 